In [ ]:
from pathlib import Path
import ibis
from ibis import _
from neo.helpers import normalize_stock_code

# 查找 pyproject.toml 文件，来确定项目根目录
current_dir = Path(__name__).resolve().parent
project_dir = current_dir
while project_dir != project_dir.parent:
    if (project_dir / "pyproject.toml").exists():
        break
    project_dir = project_dir.parent
else:
    raise FileNotFoundError("Could not find pyproject.toml in any parent directory")

DB_PATH = project_dir / "data/metadata.db"
print(DB_PATH)
con = ibis.duckdb.connect(database=DB_PATH, read_only=True)

# 查看所有表
all = con.list_tables()
print(all)


# 检查某股票的数据量
def check_stock_data(con, table_name, stock_code):
    table = con.table(table_name)
    if "ts_code" in table.columns and stock_code:
        stock_code = normalize_stock_code(stock_code)
        expr = table.filter(_.ts_code == stock_code)
        count = expr.count().execute()
    else:
        count = table.count().execute()

    print(f"股票 {stock_code}_{table_name} 有 {count} 条数据")


symbol = "600519"

for t_name in all:
    check_stock_data(con, t_name, symbol)

con.disconnect()

/Users/yapex/workspace/stock_downloader/data/metadata.db


IOException: IO Error: Could not set lock on file "/Users/yapex/workspace/stock_downloader/data/metadata.db": Conflicting lock is held in /opt/homebrew/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/Resources/Python.app/Contents/MacOS/Python (PID 47230) by user yapex. See also https://duckdb.org/docs/stable/connect/concurrency